In [48]:
# Import libraries
from datetime import timedelta
import os
from google.colab import files

import numpy as np
import pandas as pd
from pandas_gbq import read_gbq
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import psycopg2
import time
from collections import defaultdict
import tqdm
import pylab as pl
import math
from scipy.stats import mannwhitneyu

# Make pandas dataframes prettier
from IPython.display import display, HTML, Image
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [2]:
# authenticate
auth.authenticate_user()

In [3]:
# Set up environment variables
project_id = 'my-project-1532744781277'
if project_id == 'CHANGE-ME':
  raise ValueError('You must change project_id to your GCP project.')
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

#Users should have their own project_id or import

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# set the dataset
# if you want to use the demo, change this to mimic_demo
hosp_dataset_4 = 'mimiciv_3_1_hosp'
icu_dataset_4 = 'mimiciv_3_1_icu'
derived_dataset_4 = 'mimiciv_3_1_derived'
derived_dataset_3 = 'mimiciii_derived'
clinical_dataset_3 = 'mimiciii_clinical'



#indicate whether to run a limited sample size for testing purposes
limited_sample = False


# test it works
df = run_query("""
SELECT subject_id
FROM `physionet-data.mimiciv_3_1_hosp.patients`
WHERE subject_id = 10012853
""")
assert df.shape[0] == 1, 'unable to query MIMIC-IV!'

Downloading: 100%|██████████|


In [13]:
query = """
-- Identify The presence of a mechanical ventilation using settings
select
  stay_id, charttime
  -- case statement determining whether it is an instance of mech vent
  , max(
    case
      when itemid is null or value is null then 0 -- can't have null values
      when itemid = 720 and value != 'Other/Remarks' THEN 1  -- VentTypeRecorded
      when itemid = 223848 and value != 'Other' THEN 1
      when itemid = 223849 then 1 -- ventilator mode
      when itemid = 467 and value = 'Ventilator' THEN 1 -- O2 delivery device == ventilator
      when itemid in
        (
        445, 448, 449, 450, 1340, 1486, 1600, 224687 -- minute volume
        , 639, 654, 681, 682, 683, 684,224685,224684,224686 -- tidal volume
        , 218,436,535,444,459,224697,224695,224696,224746,224747 -- High/Low/Peak/Mean/Neg insp force ("RespPressure")
        , 221,1,1211,1655,2000,226873,224738,224419,224750,227187 -- Insp pressure
        , 543 -- PlateauPressure
        , 5865,5866,224707,224709,224705,224706 -- APRV pressure
        , 60,437,505,506,686,220339,224700 -- PEEP
        , 3459 -- high pressure relief
        , 501,502,503,224702 -- PCV
        , 223,667,668,669,670,671,672 -- TCPCV
        , 224701 -- PSVlevel
        )
        THEN 1
      else 0
    end
    ) as MechVent
    , max(
      case
        -- initiation of oxygen therapy indicates the ventilation has ended
        when itemid = 226732 and value in
        (
          'Nasal cannula', -- 153714 observations
          'Face tent', -- 24601 observations
          'Aerosol-cool', -- 24560 observations
          'Trach mask ', -- 16435 observations
          'High flow neb', -- 10785 observations
          'Non-rebreather', -- 5182 observations
          'Venti mask ', -- 1947 observations
          'Medium conc mask ', -- 1888 observations
          'T-piece', -- 1135 observations
          'High flow nasal cannula', -- 925 observations
          'Ultrasonic neb', -- 9 observations
          'Vapomist' -- 3 observations
        ) then 1
        when itemid = 467 and value in
        (
          'Cannula', -- 278252 observations
          'Nasal Cannula', -- 248299 observations
          -- 'None', -- 95498 observations
          'Face Tent', -- 35766 observations
          'Aerosol-Cool', -- 33919 observations
          'Trach Mask', -- 32655 observations
          'Hi Flow Neb', -- 14070 observations
          'Non-Rebreather', -- 10856 observations
          'Venti Mask', -- 4279 observations
          'Medium Conc Mask', -- 2114 observations
          'Vapotherm', -- 1655 observations
          'T-Piece', -- 779 observations
          'Hood', -- 670 observations
          'Hut', -- 150 observations
          'TranstrachealCat', -- 78 observations
          'Heated Neb', -- 37 observations
          'Ultrasonic Neb' -- 2 observations
        ) then 1
      else 0
      end
    ) as OxygenTherapy
    , max(
      case when itemid is null or value is null then 0
        -- extubated indicates ventilation event has ended
        when itemid = 640 and value = 'Extubated' then 1
        when itemid = 640 and value = 'Self Extubation' then 1
      else 0
      end
      )
      as Extubated
    , max(
      case when itemid is null or value is null then 0
        when itemid = 640 and value = 'Self Extubation' then 1
      else 0
      end
      )
      as SelfExtubated
from `physionet-data.mimiciv_3_1_icu.chartevents` ce
where ce.value is not null
and itemid in
(
    -- the below are settings used to indicate ventilation
      720, 223849 -- vent mode
    , 223848 -- vent type
    , 445, 448, 449, 450, 1340, 1486, 1600, 224687 -- minute volume
    , 639, 654, 681, 682, 683, 684,224685,224684,224686 -- tidal volume
    , 218,436,535,444,224697,224695,224696,224746,224747 -- High/Low/Peak/Mean ("RespPressure")
    , 221,1,1211,1655,2000,226873,224738,224419,224750,227187 -- Insp pressure
    , 543 -- PlateauPressure
    , 5865,5866,224707,224709,224705,224706 -- APRV pressure
    , 60,437,505,506,686,220339,224700 -- PEEP
    , 3459 -- high pressure relief
    , 501,502,503,224702 -- PCV
    , 223,667,668,669,670,671,672 -- TCPCV
    , 224701 -- PSVlevel

    -- the below are settings used to indicate extubation
    , 640 -- extubated

    -- the below indicate oxygen/NIV, i.e. the end of a mechanical vent event
    , 468 -- O2 Delivery Device#2
    , 469 -- O2 Delivery Mode
    , 470 -- O2 Flow (lpm)
    , 471 -- O2 Flow (lpm) #2
    , 227287 -- O2 Flow (additional cannula)
    , 226732 -- O2 Delivery Device(s)
    , 223834 -- O2 Flow

    -- used in both oxygen + vent calculation
    , 467 -- O2 Delivery Device
)
group by stay_id, charttime
UNION DISTINCT
-- add in the extubation flags from procedureevents_mv
-- note that we only need the start time for the extubation
-- (extubation is always charted as ending 1 minute after it started)
select
  stay_id, starttime as charttime
  , 0 as MechVent
  , 0 as OxygenTherapy
  , 1 as Extubated
  , case when itemid = 225468 then 1 else 0 end as SelfExtubated
from `physionet-data.mimiciv_3_1_icu.procedureevents`
where itemid in
(
  227194 -- "Extubation"
, 225468 -- "Unplanned Extubation (patient-initiated)"
, 225477 -- "Unplanned Extubation (non-patient initiated)"
);
"""
durations_df = run_query(query)

Downloading: 100%|██████████|


In [16]:
print(len(durations_df))
durations_df.to_csv("ventilation_classification.csv", index=False)
files.download("ventilation_classification.csv")

2470083


Users should add this dataset to their BigQuery source before proceeding

In [44]:
vent_durations_query = """
with vd0 as
(
  select
    stay_id
    -- this carries over the previous charttime which had a mechanical ventilation event
    , case
        when MechVent=1 then
          LAG(CHARTTIME, 1) OVER (partition by stay_id, MechVent order by charttime)
        else null
      end as charttime_lag
    , charttime
    , MechVent
    , OxygenTherapy
    , Extubated
    , SelfExtubated
  from `my-project-1532744781277.mimiciv_3_1_prj_vent_durations.mimiciv_3_1_prj_vent_durations`
)
, vd1 as
(
  select
      stay_id
      , charttime_lag
      , charttime
      , MechVent
      , OxygenTherapy
      , Extubated
      , SelfExtubated

      -- if this is a mechanical ventilation event, we calculate the time since the last event
      , case
          -- if the current observation indicates mechanical ventilation is present
          -- calculate the time since the last vent event
          when MechVent=1 then
            DATETIME_DIFF(CHARTTIME, charttime_lag, MINUTE)/60
          else null
        end as ventduration

      , LAG(Extubated,1)
      OVER
      (
      partition by stay_id, case when MechVent=1 or Extubated=1 then 1 else 0 end
      order by charttime
      ) as ExtubatedLag

      -- now we determine if the current mech vent event is a "new", i.e. they've just been intubated
      , case
        -- if there is an extubation flag, we mark any subsequent ventilation as a new ventilation event
          --when Extubated = 1 then 0 -- extubation is *not* a new ventilation event, the *subsequent* row is
          when
            LAG(Extubated,1)
            OVER
            (
            partition by stay_id, case when MechVent=1 or Extubated=1 then 1 else 0 end
            order by charttime
            )
            = 1 then 1
          -- if patient has initiated oxygen therapy, and is not currently vented, start a newvent
          when MechVent = 0 and OxygenTherapy = 1 then 1
            -- if there is less than 8 hours between vent settings, we do not treat this as a new ventilation event
          when CHARTTIME > DATETIME_ADD(charttime_lag, INTERVAL '8' HOUR)
            then 1
        else 0
        end as newvent
  -- use the staging table with only vent settings from chart events
  FROM vd0 ventsettings
)
, vd2 as
(
  select vd1.*
  -- create a cumulative sum of the instances of new ventilation
  -- this results in a monotonic integer assigned to each instance of ventilation
  , case when MechVent=1 or Extubated = 1 then
      SUM( newvent )
      OVER ( partition by stay_id order by charttime )
    else null end
    as ventnum
  --- now we convert CHARTTIME of ventilator settings into durations
  from vd1
)
-- create the durations for each mechanical ventilation instance
select stay_id
  -- regenerate ventnum so it's sequential
  , ROW_NUMBER() over (partition by stay_id order by ventnum) as ventnum
  , min(charttime) as starttime
  , max(charttime) as endtime
  , DATETIME_DIFF(max(charttime), min(charttime), MINUTE)/60 AS duration_hours
from vd2
group by stay_id, vd2.ventnum
having min(charttime) != max(charttime)
-- patient had to be mechanically ventilated at least once
-- i.e. max(mechvent) should be 1
-- this excludes a frequent situation of NIV/oxygen before intub
-- in these cases, ventnum=0 and max(mechvent)=0, so they are ignored
and max(mechvent) = 1
order by stay_id, ventnum
"""

vent_durations_df = run_query(vent_durations_query)

Downloading: 100%|██████████|


In [49]:
print(len(vent_durations_df))
vent_durations_df.to_csv("ventilation_durations_df.csv", index=False)
files.download("ventilation_durations_df.csv")

54902


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Users should add this dataset to their BigQuery source before proceeding

In [54]:
vaso_query = """
-- Consecutive administrations are numbered 1, 2, ...
-- Total time on the drug can be calculated from this table
-- by grouping using stay_id
-- select only the ITEMIDs from the inputevents_mv table related to vasopressors
WITH io_mv AS (
  SELECT
    stay_id, linkorderid, starttime, endtime
  FROM `physionet-data.mimiciv_3_1_icu.inputevents`
  WHERE itemid IN (
    221906,221289,221749,222315,221662,221653,221986
  )
  AND statusdescription != 'Rewritten'
),
vasomv AS (
  SELECT
    stay_id, linkorderid,
    MIN(starttime) AS starttime,
    MAX(endtime) AS endtime
  FROM io_mv
  GROUP BY stay_id, linkorderid
),
joined_intervals AS (
  SELECT
    s1.stay_id,
    s1.starttime,
    t1.endtime
  FROM vasomv s1
  JOIN vasomv t1
    ON s1.stay_id = t1.stay_id
    AND s1.starttime <= t1.endtime
),
vasomv_grp AS (
  SELECT
    s1.stay_id,
    s1.starttime,
    MIN(t1.endtime) AS endtime
  FROM joined_intervals t1
  JOIN vasomv s1
    ON s1.stay_id = t1.stay_id AND s1.starttime = t1.starttime
  WHERE NOT EXISTS (
    SELECT 1 FROM vasomv t2
    WHERE t1.stay_id = t2.stay_id
      AND t1.endtime >= t2.starttime
      AND t1.endtime < t2.endtime
  )
  AND NOT EXISTS (
    SELECT 1 FROM vasomv s2
    WHERE s1.stay_id = s2.stay_id
      AND s1.starttime > s2.starttime
      AND s1.starttime <= s2.endtime
  )
  GROUP BY s1.stay_id, s1.starttime
)

SELECT
  stay_id,
  ROW_NUMBER() OVER (PARTITION BY stay_id ORDER BY starttime) AS vasonum,
  starttime,
  endtime,
  DATETIME_DIFF(endtime, starttime, HOUR) AS duration_hours
FROM vasomv_grp
ORDER BY stay_id, vasonum;

"""
vaso_df = run_query(vaso_query)

Downloading: 100%|██████████|


In [56]:
print(len(vaso_df))
vaso_df.to_csv("vasopressor_df.csv", index=False)
files.download("vasopressor_df.csv")

53842


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Users should add this dataset to their BigQuery source before proceeding